In [2]:
import numpy as np
import cv2
from test import *

In [3]:
# Static values
PATH = 'testVideos/IMG_9367.MOV'

#how many pixel to play until pause
SHOW_PIXEL = 1


In [3]:
#loop over all frames  'space' = next frame | 'q' = quit
cap = cv2.VideoCapture(PATH)
if (cap.isOpened()== False):
    print("Error opening video stream or file")

total_frames = 0
while cap.isOpened():
    works, frame = cap.read()
    #if vidoe is not done do
    if works == True:

        #when you reach the pause frame do
        if((total_frames % SHOW_PIXEL) == 0):
            #Caption 'frame'
            cv2.imshow('Video', frame)

            #press 'E' to get next frame
            if(cv2.waitKey(5000) == ord('e')):
                continue


        if(cv2.waitKey(25) == ord('q')):
                break
        
        total_frames += 1

    else:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [4]:
def captionFrame(frame):
    pass